# Fraud Detection System Report

## System Design
- How your system design meets the requirements gathered.
- Use accompanying diagrams to describe your system components and processes.

## Data, Data Pipelines, and Model
- Description of the data.
- Detailed explanation of data pipelines and model.

## Metrics Definition
- Provide a detailed description of your offline and online metrics and their purpose.

## Analysis on System parameters and Configurations
- Dataset evaluation.
- Feature engineering evaluation.
- Model evaluation.

## Post-deployment Policies
- Monitoring and maintenance plan.
- Fault mitigation strategies.